# Extraction

For all videos in a dataset and for all landmarks, extract the RGB and BVP signals and evaluate the result.
Save the result in a dataframe.

In [1]:
%load_ext autoreload
%autoreload 2

import pyVHR as vhr
import numpy as np
from pyVHR.analysis.pipelineLandmarks import *
import plotly.express as px

import constants
import pandas as pd
import numpy as np
import pandas as pd
from tqdm import tqdm

vhr.plot.VisualizeParams.renderer = 'vscode' 

C:\Users\erolland\AppData\Roaming\Python\Python39\site-packages\cupy\_environment.py:487: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda11x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
c:\Users\erolland\.conda\envs\project\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to l

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



# Extract RGB

- Extract windowed RGB traces for all landmarks

In [59]:
## Args
dataset_name = 'mr_nirp' 
methods = ['cupy_CHROM', 'cpu_LGI']
roi_approach = 'landmark'
sampling_method = 'random'
nb_sample_points = 2000
seconds = 60
winsize = 10      
stride = 10
cuda = True

pipeline = PipielineLandmarks()
dataset = pipeline.get_dataset(dataset_name)
allvideo = dataset.videoFilenames
all_landmarks = list(vhr.extraction.utils.CustomLandmarks().get_all_landmarks().keys())
print(f"all_landmarks: {all_landmarks}")

all_landmarks: ['lower_medial_forehead', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead', 'glabella', 'upper_nasal_dorsum', 'lower_nasal_dorsum', 'soft_triangle', 'left_ala', 'right_ala', 'nasal_tip', 'left_lower_nasal_sidewall', 'right_lower_nasal_sidewall', 'left_mid_nasal_sidewall', 'right_mid_nasal_sidewall', 'philtrum', 'left_upper_lip', 'right_upper_lip', 'left_nasolabial_fold', 'right_nasolabial_fold', 'left_temporal', 'right_temporal', 'left_malar', 'right_malar', 'left_lower_cheek', 'right_lower_cheek', 'chin', 'left_marionette_fold', 'right_marionette_fold']


In [60]:
def add_data(landmark, videoFileName, windowed_sig, timesES):
  """
  Add data to the dictionary
  """
  
  data = {}
  data['dataset'] = dataset_name
  data['videoIdx'] = allvideo.index(videoFileName)
  data['videoFilename'] = constants.get_subject_name(dataset_name, videoFileName)
  data['landmarks'] = landmark # single landmark
  data['rPPG'] = windowed_sig
  data['timesES'] = timesES

  return data

def reject_video(dataset_name, videoFileName):
  """
  Reject videos that are not considered in the dataset
  Args:
    dataset_name: str
    videoFileName: str
  """
  if dataset_name == 'ubfc_phys' and 'T1' not in videoFileName:
    return True
  if dataset_name == 'mr_nirp' and 'still' not in videoFileName:
    return True
  for subject in constants.eliminated_subjects:
    if subject in videoFileName:
      return True
  return False

In [62]:
datas = []

for videoIdx in tqdm(range(len(allvideo[:2]))):
    fname = dataset.getSigFilename(videoIdx)
    videoFileName = dataset.getVideoFilename(videoIdx)
    if reject_video(dataset_name, videoFileName):
        continue
    print(videoIdx, videoFileName)

    for landmark in all_landmarks[:2]:
        try:
            windowed_sig, timesES = pipeline.extract_rgb([landmark], videoFileName, roi_approach, sampling_method, nb_sample_points, 
                                                         seconds, winsize, stride, verb=False, cuda=cuda)
            data = add_data(landmark, videoFileName, windowed_sig, timesES)
            datas.append(data)
        except Exception as e:
            print("Error in landmark ", landmark, " for video ", videoFileName)
            print(e)
            pass

  0%|          | 0/2 [00:00<?, ?it/s]

1 D:/datasets_rppg/MR-NIRP_indoor\Subject1_still_940-015\Subject1_still_940\RGB_corrected\Subject1_still_940.avi


100%|██████████| 2/2 [03:04<00:00, 92.29s/it]


In [63]:
df = pd.DataFrame(datas)
df['sampling'] = sampling_method + '_' + str(winsize) + '_' + str(stride)
df['timesES'] = [timesES for i in df.index]
df.head()

,dataset,videoIdx,videoFilename,landmarks,rPPG,timesES,sampling
0,mr_nirp,1,Subject1_still_940,lower_medial_forehead,[[[[99.857 99.021 99.1715 98.8615 98.9175 99...,"[5.0, 15.0, 25.0, 35.0, 45.0, 55.0]",random_10_10
1,mr_nirp,1,Subject1_still_940,left_lower_lateral_forehead,[[[[83.0895 84.4545 82.879 83.1045 83.65 83...,"[5.0, 15.0, 25.0, 35.0, 45.0, 55.0]",random_10_10


In [11]:
# df.to_hdf(f'../result/{dataset_name.upper()}/{dataset_name.upper()}_rPPG.h5', key='df', mode='w')

C:\Users\erolland\AppData\Local\Temp\3\ipykernel_22328\301974329.py:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['dataset', 'videoFilename', 'landmark', 'bpmGT', 'rPPG', 'timesGT',
       'timesES', 'sampling'],
      dtype='object')]

  df.to_hdf(f'../results/brightness_roi/lgi/{dataset_name.upper()}/{dataset_name.upper()}_rPPG_talk_60s.h5', key='df', mode='w')


# Extract BVP

In [2]:
## Args
dataset_name = 'mr_nirp' 
methods = ['cupy_CHROM', 'cpu_LGI']
roi_approach = 'landmark'
sampling_method = 'random'
nb_sample_points = 2000
seconds = 60
winsize = 10      
stride = 10
cuda = True

pipeline = PipielineLandmarks()
dataset = pipeline.get_dataset(dataset_name)
allvideo = dataset.videoFilenames
custom_landmarks = vhr.extraction.utils.CustomLandmarks()
all_landmarks_names = list(custom_landmarks.get_all_landmarks().keys())
videoFPS, sigFPS = constants.get_fps(dataset_name)

overlap = winsize - stride
length = int((seconds - winsize) / (winsize - overlap)) + 1 # Number of extracted windows
print(f"Stride: {stride}, Winsize: {winsize}, Duration: {seconds}, Length: {length}")

Stride: 10, Winsize: 10, Duration: 60, Length: 6


In [3]:
# Top landmarks: Landmarks considered in combinations
top_ldmks = ['glabella', 'upper_nasal_dorsum', 'lower_medial_forehead', 'soft_triangle', 'malar', 'lower_lateral_forehead', 'nasal_tip', 'chin', 'lower_cheek', 'ala', 'nasolabial_fold', 'marionette_fold']
print(len(top_ldmks), top_ldmks)

# top_ldmks_sym : landmarks with left and right component as separate
ldmk_names_dict = {ldmk.replace('left_', '').replace('right_', ''): ldmk for ldmk in all_landmarks_names} # name without left right: name with right
top_ldmks_sym = [ldmk_names_dict[ldmk] for ldmk in top_ldmks]
for ldmk in top_ldmks_sym:
    if 'right_' in ldmk:
        top_ldmks_sym.append(ldmk.replace('right', 'left'))
print(len(top_ldmks_sym), top_ldmks_sym)

12 ['glabella', 'upper_nasal_dorsum', 'lower_medial_forehead', 'soft_triangle', 'malar', 'lower_lateral_forehead', 'nasal_tip', 'chin', 'lower_cheek', 'ala', 'nasolabial_fold', 'marionette_fold']
18 ['glabella', 'upper_nasal_dorsum', 'lower_medial_forehead', 'soft_triangle', 'right_malar', 'right_lower_lateral_forehead', 'nasal_tip', 'chin', 'right_lower_cheek', 'right_ala', 'right_nasolabial_fold', 'right_marionette_fold', 'left_malar', 'left_lower_lateral_forehead', 'left_lower_cheek', 'left_ala', 'left_nasolabial_fold', 'left_marionette_fold']


In [19]:
# Landmarks choice
case = 'combine'
cases = ['ind_28', 'ind_18', 'combine']
if case == 'combine':
    all_landmarks = custom_landmarks.get_landmarks(case, min_len=2, max_len=5, all_landmarks_names=top_ldmks_sym)
else:
    all_landmarks = custom_landmarks.get_landmarks(case)
print(f"Chosen_landmarks: {len(all_landmarks)}")

Chosen_landmarks: 1573


In [11]:
# Read extracted RGB signals
df = pd.read_hdf(f'../result/{dataset_name.upper()}/{dataset_name.upper()}_rPPG.h5', key='df')

print(f"Single landmark: {df['landmark'].unique().size}")
print(f"Dataset {df['dataset'].unique()} with {df['videoFilename'].unique().size} files" )
print("Sampling: ",  df['sampling'].unique()) 
print("Extracted video lenghth: ", len(df.loc[0, 'rPPG']) / videoFPS, "seconds")
df.head()

Single landmark: 28
Dataset ['mr_nirp'] with 8 files
Sampling:  ['random_10_10']
Extracted video lenghth:  0.2 seconds


,dataset,videoIdx,videoFilename,landmark,rPPG,timesES,sampling
0,mr_nirp,1,Subject1_still_940,lower_medial_forehead,[[[[99.8145 99.135 99.0145 99.036 98.789 99...,"[5.0, 15.0, 25.0, 35.0, 45.0, 55.0]",random_10_10
1,mr_nirp,1,Subject1_still_940,left_lower_lateral_forehead,[[[[82.3205 83.601 82.9905 83.351 83.4875 83...,"[5.0, 15.0, 25.0, 35.0, 45.0, 55.0]",random_10_10
2,mr_nirp,1,Subject1_still_940,right_lower_lateral_forehead,[[[[67.4085 65.7125 65.085 66.156 65.9555 66...,"[5.0, 15.0, 25.0, 35.0, 45.0, 55.0]",random_10_10
3,mr_nirp,1,Subject1_still_940,glabella,[[[[102.7315 101.3095 100.939 101.723 101.22...,"[5.0, 15.0, 25.0, 35.0, 45.0, 55.0]",random_10_10
4,mr_nirp,1,Subject1_still_940,upper_nasal_dorsum,[[[[72.413 71.8355 71.229 72.01 71.951 71...,"[5.0, 15.0, 25.0, 35.0, 45.0, 55.0]",random_10_10


In [21]:
# Extract BVP and evaluate results

res = pyVHR.analysis.pipelineLandmarks.TestResult()

for videoIdx in tqdm(df.videoIdx.unique()[:1]):
    try:
        ### Get PPG signal data
        PPG_win, bpmGT, timesGT, videoFileName = pipeline.get_signal_data(videoIdx, dataset)
    except Exception as e:
        print(f"Error in {videoIdx} {videoFileName}: {e}")
        continue

    for landmarks in all_landmarks[:1]:
        try:    
            timesES = df['timesES'].iloc[0]

            rppgs = df.loc[(df['videoFilename'] == videoFileName) & (df['landmark'].isin(landmarks)), 'rPPG',].to_numpy()
            windowed_sig = np.concatenate([rppg for rppg in rppgs], axis=1)            
            windowed_sig = windowed_sig[:int(seconds/winsize)] # only take seconds length, here 60s
            timesES = timesES[:int(seconds/winsize)] # only take seconds length, here 60s

            for method in methods:
                bvps_win, timesES, bpmES = pipeline.extract_bpm(windowed_sig, timesES, videoFPS, roi_approach, method,winsize=winsize, cuda=cuda)
                res = pipeline.evaluate_extraction(bvps_win, bpmES, bpmGT, timesES, timesGT, PPG_win, videoFPS, res, method, videoFileName, landmarks)
        except Exception as e:
            print(f"Error in {videoIdx} {videoFileName}: {e}")
            continue
    

100%|██████████| 1/1 [00:03<00:00,  3.82s/it]


In [29]:
# Format the dataframe for results
df_res = res.dataFrame
df_res['dataset'] = dataset_name
df_res['sampling'] = sampling_method + '_' + str(winsize) + '_' + str(stride)

# More readabale videoFilename and videoIdx
df_res['videoFilename'] = df_res['videoFilename'].apply(lambda x: constants.get_subject_name(dataset_name, x))
indexes = {i: constants.get_subject_name(dataset_name, video) for i, video in enumerate(allvideo)}
indexes = pd.DataFrame({'videoIdx':list(indexes.keys()), 'videoFilename':list(indexes.values())})
df_res = df_res.drop(columns=['videoIdx'])
df_res.insert(3, 'videoIdx', df_res['videoFilename'].map(indexes.set_index('videoFilename')['videoIdx']))

# Map the landmarks to the corresponding face regions
face_regions = pyVHR.analysis.CustomLandmarks().get_face_regions()
# for roi in list(face_regions.keys()):
#     df_res.loc[df_res['landmarks'].apply(lambda x: x[0]).isin(face_regions[f'{roi}']),'ROI'] = roi
if case == 'ind_18' or case == 'ind_28': 
  for face_region in list(face_regions.keys()):
      df_res.loc[df_res['landmarks'].apply(lambda x: x[0]).isin(face_regions[f'{face_regions}']),'region'] = face_region
if case == 'combine':
  ldmk_roi = {landmark: face_region for face_region, landmarks in face_regions.items() for landmark in landmarks}
  df_res['region'] = df_res['landmarks'].apply(lambda x: '_'.join(set([ldmk_roi[landmark] for landmark in x])))


print(f"Landmark: {df_res['landmarks'].unique().size}")
print(f"Dataset {df_res['dataset'].unique()} with {df_res['videoFilename'].unique().size} files" )
df_res.head()

Landmark: 1
Dataset ['mr_nirp'] with 1 files


,method,dataset,videoFilename,videoIdx,landmarks,RMSE,MAE,PCC,CCC,SNR,MAX,MAD,rPPG_PCC,DTW,bpmGT,bpmES,timeGT,timeES,sampling,region
0,cupy_CHROM,mr_nirp,Subject1_still_940,1,"(right_lower_lateral_forehead, left_malar, rig...",0.464733,0.354167,0.934642,0.934397,NaN,1.002930,,-0.112263,1.960439,"[76.5, 76.5, 76.5, 76.5, 76.5, 76.0, 76.0, 76....","[75.8056640625, 77.1240234375, 79.3212890625, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[5.0, 15.0, 25.0, 35.0, 45.0, 55.0]",random_10_10,cheeks_forehead
1,cpu_LGI,mr_nirp,Subject1_still_940,1,"(right_lower_lateral_forehead, left_malar, rig...",0.585209,0.477702,0.913093,0.906656,NaN,1.222656,,0.131464,1.748074,"[76.5, 76.5, 76.5, 76.5, 76.5, 76.0, 76.0, 76....","[75.5859375, 77.1240234375, 79.7607421875, 78....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[5.0, 15.0, 25.0, 35.0, 45.0, 55.0]",random_10_10,cheeks_forehead


In [34]:
df.head(1)

,dataset,videoIdx,videoFilename,method,landmarks,bpmGT,bpmES,timeGT,timeES,BVP_win,RMSE,MAE,PCC,timePCC,timeDTW,SNR,ROI,config
0,mr_nirp,1,Subject1_still_940,cupy_CHROM,"(lower_medial_forehead,)","[76.5, 76.5, 76.5, 76.5, 76.5, 76.0, 76.0, 76....","[75.5859375, 76.46484375, 79.1015625, 78.22265...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[5.0, 15.0, 25.0, 35.0, 45.0, 55.0]","[[[-0.0008787291958837653, -0.162179957684393,...",0.801997,0.68099,0.863737,0.375969,2.208592,[0.46733564231544733],forehead,2000_win10-0


In [ ]:
path = f'../result/{dataset_name.upper()}/'
filenames = {'ind_28':'28', 'ind_18':'18', 'combine':'combine'}
filename = path+f'{dataset_name.upper()}_{filenames[case]}.h5'

print(filename)
# df_res.to_hdf(filename, key='df', mode='w')